In [4]:
import operator
import os
from typing import Annotated, List, TypedDict

from langchain_chroma import Chroma
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, AnyMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel
from langgraph.checkpoint import BaseCheckpointSaver
from langgraph.graph import StateGraph

PLAN_PROMPT = """You are an expert Business Analyst tasked with writing a high level outline of a business report. \
Write such an outline for the user provided question. Give an outline of the report along with any relevant notes \
or instructions for the sections."""

RESEARCH_PROMPT = """You are a data analyst with years of experience of the bank's database. 
you have a tool that can find information based on your query. 
Generate a list of search queries that will gather any relevant information. Only generate 2 queries max."""

DEBUG = os.environ.get("DEBUG")


class Queries(BaseModel):
    queries: List[str]


class AgentState(TypedDict):
    question: str
    sql_dialect: str
    need_sql_code: bool
    sql_code: str
    docs: List[str]
    answer: str
    history: Annotated[list[AnyMessage], operator.add]


class SQLAgent:

    def __init__(
        self,
        model: BaseChatModel,
        checkpointer: BaseCheckpointSaver,
        vector_db: Chroma,
    ):
        self.model = model
        self.checkpointer = checkpointer
        self.vector_db = vector_db
        builder = StateGraph(AgentState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("research", self.research_node)
        builder.add_node("planner", self.plan_node)
        builder.add_node("generate_sql_code", self.generate_sql_code_node)
        builder.add_node("generate_answer", self.answer_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "research")
        builder.add_edge("research", "planner")
        builder.add_conditional_edges(
            "planner",
            self.should_generate_sql_code,
            {True: "generate_sql_code", False: "generate_answer"},
        )
        builder.add_edge("generate_sql_code", "generate_answer")
        self.graph = builder.compile(checkpointer=checkpointer, debug=DEBUG)

    def initialize_node(self, state: AgentState):

        return {"answer": "", "sql_code": "", "need_sql_code": False}

    def research_node(self, state: AgentState):
        """
        使用embedding來找到相關的文檔的片段, 並將完整文檔傳入
        """
        retrieved_docs = self.vector_db.similarity_search(state["question"], k=4)
        print(len(retrieved_docs), [doc.metadata for doc in retrieved_docs])
        for doc in retrieved_docs:
            print(doc.metadata["filename"])
        table_names = set([doc.metadata["table_name"] for doc in retrieved_docs])
        print(f"retrieved tables: {table_names}")
        result = []
        for table_name in table_names:
            docs = self.vector_db.get(where={"table_name": table_name})["documents"]
            result.extend(docs)
        return {"docs": result}

    def plan_node(self, state: AgentState):
        docs = "\n\n".join(state["docs"])
        messages = [
            SystemMessage(
                content="You are a Business Analyst. Identify if you need SQL code to answer the question. reply y if needed else n"
            ),
        ]
        messages.extend(state["history"])
        messages.extend(
            [
                AIMessage(content=docs),
                HumanMessage(content=f'question: {state["question"]}'),
            ]
        )
        response = self.model.invoke(messages)
        need_sql_code = "y" in response.content
        return {"need_sql_code": need_sql_code}

    def should_generate_sql_code(self, state: AgentState):
        if state["need_sql_code"]:
            return True
        return False

    def generate_sql_code_node(self, state: AgentState):
        docs = "\n\n".join(state["docs"])
        dialect = state["sql_dialect"]
        messages = [
            SystemMessage(
                content=f"You are a {dialect} expert. Write a SQL code to solve the quesion. You can only use the information below."
            ),
            AIMessage(content=docs),
            HumanMessage(content=f'question: {state["question"]}'),
        ]
        response = self.model.invoke(messages)
        return {"sql_code": response.content}

    def answer_node(self, state: AgentState):
        docs = "\n\n".join(state["docs"])
        history = state["history"]

        if state["need_sql_code"]:
            messages = [
                SystemMessage(
                    content=f"You are a friendly business analyst. Answer the question with the following context. The answer should come with the SQL code"
                )
            ]
            messages.extend(history)
            messages.extend(
                [
                    AIMessage(
                        content=f"I have done the research on the question, and here is the documents retrieved: {docs}"
                    ),
                    AIMessage(content=f'SQL code: {state["sql_code"]}'),
                    HumanMessage(content=f'question: {state["question"]}'),
                ]
            )
        else:
            messages = [
                SystemMessage(
                    content=f"You are a friendly business analyst. Answer the question with the following context. The answer doesn't need to come with the SQL code"
                )
            ]
            messages.extend(history)
            messages.extend(
                [
                    AIMessage(content=docs),
                    HumanMessage(content=f'question: {state["question"]}'),
                ]
            )
        response = self.model.invoke(messages)
        return {
            "answer": response.content,
            "history": [
                HumanMessage(content=state["question"]),
                AIMessage(content=response.content),
            ],
        }


In [3]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langgraph.checkpoint.sqlite import SqliteSaver

from ai_helper.src.agents import SQLAgent

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")
# 建立測試資料
table_docs = [
    Document(
        page_content="""[
{
    "name":"cust_id",
    "type": "STRING",
    "description":"客戶的id"
},
{
    "name":"age_col",
    "type": "DOUBLE",
    "description":"客戶的年齡"
}
]""",
        metadata={
            "filename": "customer_table.json",
            "table_name": "customer_table",
            "part": "columns",
        },
    ),
    Document(
        page_content="""{
"table_name":"customer_table",
"database_name":"datamart_db",
"description":"This table stores all customer's personal information."
}""",
        metadata={
            "filename": "customer_table.json",
            "table_name": "customer_table",
            "part": "main",
        },
    ),
]

vector_db = Chroma.from_documents(
    table_docs, GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
)
bot = SQLAgent(model=model, vector_db=vector_db, checkpointer=memory)
thread = {"configurable": {"thread_id": "1"}}
result = bot.graph.invoke(
    {
        "question": "我是誰 ",
        "sql_dialect": "impala",
    },
    thread,
)

c:\Users\user\AppData\Local\pypoetry\Cache\virtualenvs\senpai-m2n-YTg9-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


2 [{'filename': 'customer_table.json', 'part': 'columns', 'table_name': 'customer_table'}, {'filename': 'customer_table.json', 'part': 'main', 'table_name': 'customer_table'}]
customer_table.json
customer_table.json
retrieved tables: {'customer_table'}


In [5]:
result

{'question': '我是誰 ',
 'sql_dialect': 'impala',
 'need_sql_code': False,
 'sql_code': '',
 'docs': ['[\n{\n    "name":"cust_id",\n    "type": "STRING",\n    "description":"客戶的id"\n},\n{\n    "name":"age_col",\n    "type": "DOUBLE",\n    "description":"客戶的年齡"\n}\n]',
  '{\n"table_name":"customer_table",\n"database_name":"datamart_db",\n"description":"This table stores all customer\'s personal information."\n}'],
 'answer': '根據提供的資訊，我無法得知「你是誰」。 \n\n這些資訊描述了資料庫中一個資料表的結構和內容，包括：\n\n* **cust_id**: 客戶的唯一識別碼\n* **age_col**: 客戶的年齡\n* **table_name**: 資料表名稱為 customer_table\n* **database_name**: 資料庫名稱為 datamart_db\n* **description**:  資料表的描述\n\n這些資訊僅僅是關於客戶資料的結構和組織，並沒有包含任何可以識別你是誰的資訊。 \n\n請提供更多上下文或資訊，以便我更好地理解你的問題，並給出準確的答案。 😊 \n',
 'history': [HumanMessage(content='我是誰 '),
  AIMessage(content='根據提供的資訊，我無法得知「你是誰」。 \n\n這些資訊描述了資料庫中一個資料表的結構和內容，包括：\n\n* **cust_id**: 客戶的唯一識別碼\n* **age_col**: 客戶的年齡\n* **table_name**: 資料表名稱為 customer_table\n* **database_name**: 資料庫名稱為 datamart_db\n* **description**:  資料表的描述\n\

In [21]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str
    configurable: dict

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化狀態
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客戶特徵摘要
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="Extract a 50-character summary of the customer features."),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content[:50]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基於客戶特徵生成個人化新聞摘要
        messages = [
            SystemMessage(content="Generate a 100-character personalized news summary based on the customer features."),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"Customer features: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content[:100]}

    def investment_advice_node(self, state: CustomerState):
        # 綜合生成投資建議
        messages = [
            SystemMessage(content="Provide investment advice based on the customer features and the personalized news summary."),
            AIMessage(content=f"Customer features: {state['extracted_features']}"),
            AIMessage(content=f"News summary: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        return {"investment_advice_result": response.content}

    def output_results_node(self, state: CustomerState):
        # 返回最終結果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 輸入新聞和客戶特徵
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 年收入20萬美元。",
        "客戶B, 年齡28歲, 職業為數據分析師, 年收入10萬美元。",
        # 可以繼續添加更多客戶特徵
    ],
    "configurable": {"thread_id": "unique_thread_id_1234"}  # 唯一的 thread_id
}

# 執行圖形流程
result = bot.graph.invoke(state)

# 輸出結果
print("客戶特徵摘要:", result["extracted_features"])
print("個人化新聞摘要:", result["news_summary"])
print("未來投資建議:", result["investment_advice_result"])


ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'thread_ts']

In [24]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str
    thread_id: str

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化狀態
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客戶特徵摘要
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="Extract a 50-character summary of the customer features."),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content[:50]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基於客戶特徵生成個人化新聞摘要
        messages = [
            SystemMessage(content="Generate a 100-character personalized news summary based on the customer features."),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"Customer features: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content[:100]}

    def investment_advice_node(self, state: CustomerState):
        # 綜合生成投資建議
        messages = [
            SystemMessage(content="Provide investment advice based on the customer features and the personalized news summary."),
            AIMessage(content=f"Customer features: {state['extracted_features']}"),
            AIMessage(content=f"News summary: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        return {"investment_advice_result": response.content}

    def output_results_node(self, state: CustomerState):
        # 返回最終結果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 定義一個唯一的 thread_id
thread_id = "unique_thread_id_1234"

# 輸入新聞和客戶特徵
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 年收入20萬美元。",
        "客戶B, 年齡28歲, 職業為數據分析師, 年收入10萬美元。",
        # 可以繼續添加更多客戶特徵
    ],
    "thread_id": thread_id  # 唯一的 thread_id
}

# 執行圖形流程
result = bot.graph.invoke(state)

# 輸出結果
print("客戶特徵摘要:", result["extracted_features"])
print("個人化新聞摘要:", result["news_summary"])
print("未來投資建議:", result["investment_advice_result"])


ValueError: Checkpointer requires one or more of the following 'configurable' keys: ['thread_id', 'thread_ts']

In [8]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

# 定义状态字典类型
class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化状态
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客户特征摘要，使用中文
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="請用中文回答。擷取客户特徵並給予150字左右摘要。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content[:150]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基于客户特征生成个性化新闻摘要，使用中文
        messages = [
            SystemMessage(content="請依照提供內容給予200字內新聞摘要。"),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content[:200]}

    def investment_advice_node(self, state: CustomerState):
        # 综合生成投资建议，使用中文
        messages = [
            SystemMessage(content="請根據客戶特徵及新聞摘要給予投資建議。"),
            AIMessage(content=f"客户特徵: {state['extracted_features']}"),
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        return {"investment_advice_result": response.content}

    def output_results_node(self, state: CustomerState):
        # 返回最终结果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和检查点保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 定义一个唯一的 thread_id
thread = {"configurable": {"thread_id": "1"}}

# 输入新闻和客户特征
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 年收入20萬美元。",
        "客戶B, 年齡20歲, 職業為老師, 年收入1萬美元。",
        # 可以继续添加更多客户特征
    ]
}

# 执行图形流程
result = bot.graph.invoke(state, thread)

# 输出结果
print("客户特徵摘要:", result["extracted_features"])
print("個性化新聞摘要:", result["news_summary"])
print("投資建議:", result["investment_advice_result"])


客户特徵摘要: **客戶特徵摘要：**

客戶A為35歲的軟件工程師，年收入20萬美元，屬於高收入群體，可能對科技產品、投資理財等方面感興趣。

客戶B為20歲的老師，年收入1萬美元，屬於年輕且收入較低的群體，可能對教育培訓、個人成長以及性價比高的產品和服務更感興趣。
 

個性化新聞摘要: **市場營銷策略：**

針對客戶A，可以通過以下方式進行營銷：

* **線上廣告：** 在科技網站、投資理財平台等投放廣告，吸引對應人群。
* **內容營銷：** 製作與科技、投資相關的高質量內容，吸引潛在客戶。
* **社群媒體營銷：** 在LinkedIn等平台上建立品牌形象，與目標客戶互動。

針對客戶B，可以通過以下方式進行營銷：

* **價格策略：** 提供性價比高的產品和服務，吸
投資建議: 我需要更多信息才能提供投資建議。 請您提供以下信息：

**客戶特徵：**

* 年齡
* 風險承受能力（保守、穩健、積極）
* 投資目標（例如：退休儲蓄、子女教育、短期投機）
* 投資期限（例如：5年、10年、20年）
* 現有投資組合（如有）
* 其他任何相關信息（例如：是否有房貸、是否有其他負債）

**新聞摘要：**

請提供您想了解投資建議的新聞摘要，例如：

* 特定的公司或行業新聞
* 宏觀經濟新聞
* 政策變化

獲得以上信息後，我才能根據客戶特徵和新聞摘要提供更準確、更有價值的投資建議。 



In [65]:
# 格式化输出
def format_output_for_customer(customer_name, features, summary, advice):
    return f"### {customer_name}\n\n" \
           f"**客户特征摘要:**\n{features}\n\n" \
           f"**个性化新闻摘要:**\n{summary}\n\n" \
           f"**未来投资建议:**\n{advice}\n"

# 单独处理每个客户的信息
for i, customer_feature in enumerate(state["customer_features"]):
    # 提取单个客户特征
    state["customer_features"] = [customer_feature]

    # 执行图形流程
    result = bot.graph.invoke(state, thread)

    # 分隔每个客户的结果并输出
    customer_output = format_output_for_customer(f"客户{i + 1}", result["extracted_features"], result["news_summary"], result["investment_advice_result"])
    print(customer_output)
    print("---")  # 分隔线


生成的投资建议: 请根据客户特征和新闻摘要提供投资建议。 

请注意，我无法提供财务建议。以下建议仅供参考，不构成投资建议。

##  针对客户A的投资建议：

**目标:**  

* 年龄35岁，属于中短期投资阶段，可承受一定风险，追求资产增值。
* 年收入20万美元，属于高收入人群，有较强的风险承受能力。

**策略:**

1. **核心资产配置:** 鉴于客户A风险承受能力较强，可以考虑将大部分资金配置于风险资产，例如股票和基金，以追求更高的收益。
    * **股票**: 可以考虑配置一部分资金于科技股，但考虑到新闻中提到的科技股回调风险，建议分散投资，不要把所有鸡蛋放在一个篮子里。可以选择一些具有长期增长潜力的科技公司，例如云计算、人工智能、新能源等领域的龙头企业。
    * **基金**: 建议配置一部分资金于指数基金，例如追踪标普500指数的ETF，以分散风险，获取长期稳定的收益。
2. **卫星资产配置:** 可以考虑配置一小部分资金于避险资产，例如黄金和债券，以对冲风险。
    * **黄金**: 新闻中提到建议增持黄金，可以考虑配置一小部分资金于黄金ETF，以对冲通货膨胀和地缘政治风险。
    * **债券**: 新闻中提到建议关注美债和高评级债券，可以考虑配置一小部分资金于短期美国国债ETF，以对冲市场波动风险。
3. **现金管理**: 建议保留一部分现金，以应对突发事件和投资机会。

**具体操作建议:**

* **定期定额投资**:  可以考虑每月定投一部分资金于指数基金，以降低投资成本，分散风险。
* **逢低买入**:  可以关注新闻和市场动态，在市场回调时逢低买入优质股票和基金。
* **止盈止损**:  设定好止盈止损点，及时止盈止损，控制投资风险。

**其他建议:**

* **持续学习**:  关注市场动态，学习投资知识，不断提升投资能力。
* **寻求专业建议**:  可以咨询专业的理财规划师，根据自身情况制定个性化的投资方案。

**免责声明**:  以上建议仅供参考，不构成投资建议。投资有风险，入市需谨慎。
### 客户1

**客户特征摘要:**
客户A是一位35岁的软件工程师，年收入20万美元。

**个性化新闻摘要:**
新闻摘要: 由于日本央行意外升息，加上美国经济数据疲软，避险情绪升温，日元近期

In [6]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

# 定义状态字典类型
class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化状态
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客户特征摘要，使用中文
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="请提取客户特征摘要。请确保摘要内容完全来自以下客户特征信息，不参考其他来源。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content.strip()}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基于客户特征生成个性化新闻摘要，严格基于提供的新闻内容
        messages = [
            SystemMessage(content="请用中文回答。生成基于客户特征的200字新闻摘要。请确保摘要完全基于以下新闻内容，不参考其他来源。"),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content.strip()}

    def investment_advice_node(self, state: CustomerState):
        # 综合生成投资建议，使用中文，基于摘要和客户特征
        messages = [
            SystemMessage(content="请根据客户特征和新闻摘要提供投资建议。"),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
            AIMessage(content=f"新闻摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        
        # 添加调试信息以检查生成的内容
        print("生成的投资建议:", response.content.strip())
        
        return {"investment_advice_result": response.content.strip()}

    def output_results_node(self, state: CustomerState):
        # 返回最终结果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和检查点保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 定义一个唯一的 thread_id
thread = {"configurable": {"thread_id": "1"}}

# 输入新闻和客户特征
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 年收入20萬美元。",
        "客戶B, 年齡20歲, 職業為教師, 年收入1萬美元。",
        # 可以继续添加更多客户特征
    ]
}

# 格式化输出
def format_output_for_customer(customer_name, features, summary, advice):
    return f"### {customer_name}\n\n" \
           f"**客户特征摘要:**\n{features}\n\n" \
           f"**个性化新闻摘要:**\n{summary}\n\n" \
           f"**未来投资建议:**\n{advice}\n"

# 单独处理每个客户的信息
for i, customer_feature in enumerate(state["customer_features"]):
    # 提取单个客户特征
    state["customer_features"] = [customer_feature]

    # 执行图形流程
    result = bot.graph.invoke(state, thread)

    # 分隔每个客户的结果并输出
    customer_output = format_output_for_customer(f"客户{i + 1}", result["extracted_features"], result["news_summary"], result["investment_advice_result"])
    print(customer_output)
    print("---")  # 分隔线


生成的投资建议: ## 投资建议：

**需要更多关于客户A的信息:**

* **风险承受能力：** 客户A的风险偏好是保守、稳健还是积极？
* **投资目标：** 客户A的投资目标是什么？是短期获利、长期增值还是退休规划？
* **现有资产配置：**  客户A目前持有哪些资产？股票、债券、房地产、现金等？
* **投资经验：** 客户A是否有投资经验？对哪些金融产品比较熟悉？

**根据现有信息，初步建议：**

* **谨慎投资股市：** 目前全球股市波动剧烈，风险较高，建议客户A谨慎投资，避免追高杀低。
* **关注防御性资产：**  美债、高评级债券、医疗和公用事业类股等防御性资产受市场波动影响较小，可以作为投资组合的稳定器。
* **分散投资：**  不要把所有鸡蛋放在一个篮子里，建议客户A分散投资于不同资产类别，降低整体风险。
* **长期投资：**  投资是一个长期过程，不要被短期市场波动所左右，建议客户A制定长期投资计划并坚持执行。

**具体建议：**

* **股票：**  可以考虑逐步建仓一些优质科技股，但仓位不宜过重。
* **债券：**  建议配置一定比例的美债和高评级债券，以降低投资组合的波动性。
* **黄金：**  黄金作为避险资产，可以在市场动荡时期提供一定的保护。
* **现金：**  保留一部分现金，以便在市场出现投资机会时可以及时出手。

**最终投资建议需要根据客户A的具体情况进行调整。** 建议客户A咨询专业的理财顾问，制定个性化的投资计划。
### 客户1

**客户特征摘要:**
客戶A是一位35歲的軟體工程師，年收入20萬美元。

**个性化新闻摘要:**
新闻摘要：

受日本央行意外升息及美国经济衰退担忧影响，全球股市遭遇重挫，台股8/5更暴跌1807点，创历史最大跌幅。日圆则受避险情绪推升，兑美元一度升破145大关。分析师指出，科技股回调、中东局势升温及美国经济数据疲软等因素，导致市场恐慌情绪蔓延。建议投资人关注防御性资产，如美债、高评级债券、医疗和公用事业类股，同时可增持黄金和现金，降低投资组合风险。

**未来投资建议:**
## 投资建议：

**需要更多关于客户A的信息:**

* **风险承受能力：** 客户A的风险偏好是保守、稳健还是积极？
* **投资目标：** 客户A的投资目标是什么？是短期获利

In [22]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

# 定义状态字典类型
class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化状态
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客户特征摘要，使用中文
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="请用中文回答。提取客户特征的100字摘要。请确保摘要内容完全来自以下客户特征信息，不参考其他来源。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content[:100]}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基于客户特征生成个性化新闻摘要，严格基于提供的新闻内容
        messages = [
            SystemMessage(content="请用中文回答。生成基于客户特征的200字新闻摘要。请确保摘要完全基于以下新闻内容，不参考其他来源。"),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content[:200]}

    def investment_advice_node(self, state: CustomerState):
        # 综合生成投资建议，使用中文，基于摘要和客户特征
        messages = [
            SystemMessage(content="请根据客户特征和新闻摘要提供投资建议。请确保建议完全基于以下内容，不参考其他来源。"),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
            AIMessage(content=f"新闻摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        return {"investment_advice_result": response.content}

    def output_results_node(self, state: CustomerState):
        # 返回最终结果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和检查点保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 定义一个唯一的 thread_id
thread = {"configurable": {"thread_id": "1"}}

# 输入新闻和客户特征
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 風險承受度高，年收入20萬美元。",
        "客戶B, 年齡20歲, 職業為教師, 風險承受度低，年收入1萬美元。",
        # 可以继续添加更多客户特征
    ]
}

# 执行图形流程
result = bot.graph.invoke(state, thread)

# 输出结果
print("客户特征摘要:", result["extracted_features"])
print("个性化新闻摘要:", result["news_summary"])
print("未来投资建议:", result["investment_advice_result"])


客户特征摘要: 客户A和客户B呈现出不同的特征。A是35岁的软件工程师，高风险承受能力，年收入20万美元，是典型的风险偏好型高收入人群。B是20岁的教师，低风险承受能力，年收入1万美元，更偏向保守型投资，收入水平相对
未来投资建议: 请提供针对客户A和客户B的投资建议。



In [23]:
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import List, TypedDict

# 定义状态字典类型
class CustomerState(TypedDict):
    news_content: str
    customer_features: List[str]
    extracted_features: str
    news_summary: str
    investment_advice_result: str

class PersonalizedNewsAgent:

    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model
        self.checkpointer = checkpointer
        builder = StateGraph(CustomerState)
        builder.add_node("initialize", self.initialize_node)
        builder.add_node("extract_customer_features", self.extract_customer_features_node)
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)
        builder.add_node("investment_advice", self.investment_advice_node)
        builder.add_node("output_results", self.output_results_node)

        builder.set_entry_point("initialize")
        builder.add_edge("initialize", "extract_customer_features")
        builder.add_edge("extract_customer_features", "personalized_news_summary")
        builder.add_edge("personalized_news_summary", "investment_advice")
        builder.add_edge("investment_advice", "output_results")
        self.graph = builder.compile(checkpointer=checkpointer)

    def initialize_node(self, state: CustomerState):
        # 初始化状态
        return {
            "extracted_features": "",
            "news_summary": "",
            "investment_advice_result": "",
        }

    def extract_customer_features_node(self, state: CustomerState):
        # 提取客户特征摘要，使用中文
        customer_features_text = "\n".join(state["customer_features"])
        messages = [
            SystemMessage(content="请提取客户特征摘要。请确保摘要内容完全来自以下客户特征信息，不参考其他来源。"),
            HumanMessage(content=customer_features_text),
        ]
        response = self.model.invoke(messages)
        return {"extracted_features": response.content.strip()}

    def personalized_news_summary_node(self, state: CustomerState):
        # 基于客户特征生成个性化新闻摘要，严格基于提供的新闻内容
        messages = [
            SystemMessage(content="请用中文回答。生成基于客户特征的200字新闻摘要。请确保摘要完全基于以下新闻内容，不参考其他来源。"),
            HumanMessage(content=state["news_content"]),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
        ]
        response = self.model.invoke(messages)
        return {"news_summary": response.content.strip()}

    def investment_advice_node(self, state: CustomerState):
        # 综合生成投资建议，使用中文，基于摘要和客户特征
        messages = [
            SystemMessage(content="请根据客户特征和新闻摘要提供投资建议。"),
            AIMessage(content=f"客户特征: {state['extracted_features']}"),
            AIMessage(content=f"新闻摘要: {state['news_summary']}"),
        ]
        response = self.model.invoke(messages)
        return {"investment_advice_result": response.content.strip()}

    def output_results_node(self, state: CustomerState):
        # 返回最终结果
        return {
            "extracted_features": state["extracted_features"],
            "news_summary": state["news_summary"],
            "investment_advice_result": state["investment_advice_result"],
        }

# 初始化模型和检查点保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
memory = SqliteSaver.from_conn_string(":memory:")

# 初始化 PersonalizedNewsAgent
bot = PersonalizedNewsAgent(model=model, checkpointer=memory)

# 定义一个唯一的 thread_id
thread = {"configurable": {"thread_id": "1"}}

# 输入新闻和客户特征
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升
日圓升值，帶動亞洲多數幣別月迄今皆升值。短
線在科技股回檔及中東情勢升溫下，避險情緒主
導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政
策正常化的決心。同時宣布開啟量化緊縮，加上投
資人對於美國經濟衰退擔憂再起，風險意識升高。
導致日股急跌，原本強勢的金融類股也面臨補跌，
日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，
同時外資撒資日股，數據顯示，截至7月26日當周，
外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3
％，數據觸發「薩姆法則」預告的經濟衰退，市場
恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807
點，為史上最大跌點，收19830.88點，跌幅8.35
％，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，
財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，
台積電*(2330)法說會後股價回檔，受到美國政治
與政策不確定因素干擾，亞股當中漲幅較高的日股
及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方
面主要以先前漲多的營造、電子股跌幅較重，重點
權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%
以上。Nvidia*Blackwell量產可能延期，疊加經濟
衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢
在必行，利多美債與高評級債券。另一方面，考
量投資信心薄弱，股市下方風險仍高，衛星宜加
碼醫療與公用事業等防禦類股。同時也建議加碼
黃金、增持現金以控制投資組合波動度，降低投
資風險。""",
    "customer_features": [
        "客戶A, 年齡35歲, 職業為軟件工程師, 年收入20萬美元。",
        "客戶B, 年齡20歲, 職業為教師, 年收入1萬美元。",
        # 可以继续添加更多客户特征
    ]
}

# 格式化输出
def format_output_for_customer(customer_name, features, summary, advice):
    return f"### {customer_name}\n\n" \
           f"**客户特征摘要:**\n{features}\n\n" \
           f"**个性化新闻摘要:**\n{summary}\n\n" \
           f"**未来投资建议:**\n{advice}\n"

# 单独处理每个客户的信息
for i, customer_feature in enumerate(state["customer_features"]):
    # 提取单个客户特征
    state["customer_features"] = [customer_feature]

    # 执行图形流程
    result = bot.graph.invoke(state, thread)

    # 分隔每个客户的结果并输出
    customer_output = format_output_for_customer(f"客户{i + 1}", result["extracted_features"], result["news_summary"], result["investment_advice_result"])
    print(customer_output)
    print("---")  # 分隔线


### 客户1

**客户特征摘要:**
客户A是一位35岁的软件工程师，年收入20万美元。

**个性化新闻摘要:**
新闻摘要：

受日本央行意外加息、美国经济数据疲软以及对全球经济衰退的担忧影响，全球股市遭受重创，日圆则大幅升值。日经指数跌至半年新低，台股更是创下史上最大跌幅。科技股成为重灾区，台积电、鸿海等权重股跌幅均超过9%。避险情绪升温，美元走弱，黄金和防御性资产受到追捧。专家建议，投资者应关注美债等高评级债券，同时配置黄金和现金以降低风险。

**未来投资建议:**
请根据客户特征和新闻摘要提供投资建议。

---
### 客户2

**客户特征摘要:**
客户B是一位20岁的教师，年收入1万美元。

**个性化新闻摘要:**
受日本央行意外加息、美国经济数据疲软以及对全球经济衰退的担忧等多重因素影响，全球股市剧烈震荡，避险情绪升温，日元走强。日本股市创下半年新低，台股更是遭遇史上最大跌幅。科技股首当其冲，台积电、鸿海等权重股跌幅明显。避险资产如美债、黄金等受到追捧。建议投资者关注防御性行业，例如医疗和公用事业，同时增加现金持有以控制风险。

**未来投资建议:**
请根据客户特征和新闻摘要提供投资建议。

---


生成的投资建议: 请提供一个基于以上客户特征和新闻摘要的详细投资建议：
### 客户1

**客户特征摘要:**
客户A是一位35岁的软件工程师，年收入20万美元。

**个性化新闻摘要:**
新闻摘要：

受日本央行意外升息及美国经济衰退担忧影响，全球股市遭遇重挫，日经指数创半年新低，台股更是创下史上最大跌幅。日圆走强，亚股面临获利调节卖压，科技股首当其冲。

尽管联准会九月可能降息，但市场避险情绪浓厚，建议您关注防御性资产，例如美债、高评级债券、医疗和公用事业类股，并考虑增持黄金和现金，以降低投资组合风险。

**未来投资建议:**
请提供一个基于以上客户特征和新闻摘要的详细投资建议：

---
生成的投资建议: 请注意，以上信息仅供参考，不构成投资建议。投资有风险，请谨慎投资。

**客户B的投资建议**

考虑到客户B是一位20岁的年轻教师，年收入1万美元，以下是一些投资建议：

**目标:**

* **长期增长:**  由于客户B年轻，拥有更长的投资期限，可以承受更高的风险以寻求长期增长。
* **财富积累:**  客户B的目标应该是建立一个稳固的财务基础，为未来目标（如购房、退休）储蓄。

**风险承受能力:**

* **中等至高风险:**  年轻投资者通常有更高的风险承受能力，因为他们有更多时间从市场波动中恢复过来。

**投资组合建议:**

* **股票 (60%):**
    * **全球股票基金 (40%):**  投资于全球股票市场的低成本指数基金，例如 **Vanguard 全球股票 ETF (VT)** 或 **iShares 核心 MSCI 全球 ETF (URTH)**。
    * **新兴市场股票基金 (10%):**  投资于新兴市场的股票基金，例如 **Vanguard FTSE 新兴市场 ETF (VWO)**，以寻求更高的增长潜力，但风险也更高。
    * **主题型 ETF (10%):**  考虑投资于与长期趋势相关的主题型 ETF，例如可再生能源、人工智能或医疗保健。

* **债券 (20%):**
    * **美国国债 ETF (15%):**  投资于美国国债的低成本 ETF，例如 **iShares 核心美国国债 ETF (GOVT)**，以降低投资组合波动性。
    * **高收益债券 ETF

In [12]:
import operator
import os
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langgraph.checkpoint.sqlite import SqliteSaver

from ai_helper.src.agents import SQLAgent
from typing import Annotated, List, TypedDict

from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, AnyMessage, HumanMessage, SystemMessage
from langchain_core.pydantic_v1 import BaseModel
from langgraph.checkpoint import BaseCheckpointSaver
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
import pandas as pd

# 設置 pandas 顯示選項以顯示所有行和列
pd.set_option('display.max_columns', None)  # 顯示所有列
pd.set_option('display.max_rows', None)  # 顯示所有行
pd.set_option('display.max_colwidth', None)  # 不截斷列的內容

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議

# 定義個性化新聞代理類
class PersonalizedNewsAgent:

    # 初始化方法，創建StateGraph，並添加節點和邊
    def __init__(self, model: BaseChatModel, checkpointer: SqliteSaver):
        self.model = model  # 初始化生成式語言模型
        self.checkpointer = checkpointer  # 初始化檢查點保存器，用於保存狀態
        builder = StateGraph(CustomerState)  # 創建StateGraph並傳入狀態字典類型
        builder.add_node("initialize", self.initialize_node)  # 添加初始化節點
        builder.add_node("extract_customer_features", self.extract_customer_features_node)  # 添加客戶特徵提取節點
        builder.add_node("personalized_news_summary", self.personalized_news_summary_node)  # 添加個性化新聞摘要節點
        builder.add_node("investment_advice", self.investment_advice_node)  # 添加投資建議節點
        builder.add_node("output_results", self.output_results_node)  # 添加輸出結果節點

        builder.set_entry_point("initialize")  # 設置初始節點為“initialize”
        builder.add_edge("initialize", "extract_customer_features")  # 定義節點之間的邊，連接初始化節點和特徵提取節點
        builder.add_edge("extract_customer_features", "personalized_news_summary")  # 連接特徵提取節點和新聞摘要節點
        builder.add_edge("personalized_news_summary", "investment_advice")  # 連接新聞摘要節點和投資建議節點
        builder.add_edge("investment_advice", "output_results")  # 連接投資建議節點和輸出結果節點
        self.graph = builder.compile(checkpointer=checkpointer)  # 編譯圖結構，準備執行

    # 初始化節點，用於重置狀態
    def initialize_node(self, state: CustomerState):
        return {
            "extracted_features": "",  # 初始化提取的客戶特徵為空
            "news_summary": "",  # 初始化新聞摘要為空
            "investment_advice_result": "",  # 初始化投資建議為空
        }

    # 提取客戶特徵節點，生成客戶特徵摘要
    def extract_customer_features_node(self, state: CustomerState):
        customer_features_text = "\n".join(state["customer_features"])  # 將客戶特徵列表合併為一個字符串
        messages = [
            SystemMessage(content="請提取客戶特徵摘要。請確保摘要內容完全來自以下客戶特徵信息，不參考其他來源。"),  # 系統消息：提示只使用給定信息
            HumanMessage(content=customer_features_text),  # 輸入消息：客戶特徵文本
        ]
        response = self.model.invoke(messages)  # 調用模型生成客戶特徵摘要
        state["extracted_features"] = response.content.strip()  # 將生成的特徵摘要保存到狀態中
        print("提取的客戶特徵:", state["extracted_features"])  # 輸出調試信息
        return {"extracted_features": state["extracted_features"]}

    # 個性化新聞摘要節點，生成基於客戶特徵的新聞摘要
    def personalized_news_summary_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="請用中文回答。生成基於客戶特徵的200字新聞摘要。請確保摘要完全基於以下新聞內容，不參考其他來源。"),  # 系統消息：提示只使用給定新聞內容
            HumanMessage(content=state["news_content"]),  # 輸入消息：新聞內容
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),  # 輸入消息：提取的客戶特徵摘要
        ]
        response = self.model.invoke(messages)  # 調用模型生成新聞摘要
        state["news_summary"] = response.content.strip()  # 將生成的新聞摘要保存到狀態中
        print("生成的新聞摘要:", state["news_summary"])  # 輸出調試信息
        return {"news_summary": state["news_summary"]}

    # 投資建議節點，生成基於客戶特徵和新聞摘要的投資建議
    def investment_advice_node(self, state: CustomerState):
        messages = [
            SystemMessage(content="根據以下客戶特徵和新聞摘要，生成一個詳細的投資建議，直接給出具體的建議，大約200字內："),  # 系統消息：生成投資建議
            AIMessage(content=f"客戶特徵: {state['extracted_features']}"),  # 輸入消息：提取的客戶特徵摘要
            AIMessage(content=f"新聞摘要: {state['news_summary']}"),  # 輸入消息：生成的新聞摘要
        ]
        response = self.model.invoke(messages)  # 調用模型生成投資建議
        state["investment_advice_result"] = response.content.strip()  # 將生成的投資建議保存到狀態中
        print("生成的投資建議:", state["investment_advice_result"])  # 輸出調試信息
        return {"investment_advice_result": state["investment_advice_result"]}

    # 輸出結果節點，將所有生成的結果彙總並返回
    def output_results_node(self, state: CustomerState):
        return {
            "extracted_features": state["extracted_features"],  # 返回提取的客戶特徵
            "news_summary": state["news_summary"],  # 返回生成的新聞摘要
            "investment_advice_result": state["investment_advice_result"],  # 返回生成的投資建議
        }

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")  # 使用Google Generative AI模型
memory = SqliteSaver.from_conn_string(":memory:")  # 使用SQLite內存數據庫保存狀態

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)  # 創建個性化新聞代理實例

thread = {"configurable": {"thread_id": "1"}}  # 定義線程ID以管理多個客戶的狀態

# 定義輸入的新聞內容和客戶特徵
state = {
    "news_content": """7/31日本央行決定將基準利率調升15個基點，從
0%~0.10%升至0.15%-0.25%，並計劃每季減少
購債約4,000億日圓，升息令套利交易平倉賣壓出
籠，輔以美國7月非農數據驟降至11.4萬人與7月
ISM製造業數據低於預期，避險情緒推升美元兌日
圓8/5盤中一度跌破145大關。

在日本央行升息後，日本公債殖利率上升、推升日圓升值，帶動亞洲多數幣別月迄今皆升值。短線在科技股回檔及中東情勢升溫下，避險情緒主導料將令日圓短線持續偏強勢。

07/31 BOJ升息15個Bp至0.25%，展現朝向貨幣政策正常化的決心。同時宣布開啟量化緊縮，加上投資人對於美國經濟衰退擔憂再起，風險意識升高。導致日股急跌，原本強勢的金融類股也面臨補跌，日經指數創半年新低。

升息導致日圓急升後，外銷類股面臨沉重賣壓，同時外資撒資日股，數據顯示，截至7月26日當周，外資淨賣出6,705億日圓的日本股票。

美國7月新增非農就業遠差於預期，失業率升至4.3%，數據觸發「薩姆法則」預告的經濟衰退，市場恐慌情緒蔓延。台股5日殺聲震天，終場大跌1807點，為史上最大跌點，收19830.88點，跌幅8.35%，成交量6407.93億元。

股價提前反應基本面，第2季股價大漲者基本面佳，財報反應利多鈍化，展望樂觀仍難抵龐大獲利賣壓，台積電*(2330)法說會後股價回檔，受到美國政治與政策不確定因素干擾，亞股當中漲幅較高的日股及台股面臨獲利調節賣壓。

8/5台股出現全面性殺盤，恐慌情緒濃厚，類股方面主要以先前漲多的營造、電子股跌幅較重，重點權值-台積電、鴻海、聯發科、廣達*跌幅都超過9%以上。Nvidia*Blackwell量產可能延期，疊加經濟衰退風險，重創今年漲高的電子族群。

由於美國經濟與通膨明顯降溫，聯準會9月降息勢在必行，利多美債與高評級債券。另一方面，考量投資信心薄弱，股市下方風險仍高，衛星宜加碼醫療與公用事業等防禦類股。同時也建議加碼黃金、增持現金以控制投資組合波動度，降低投資風險。""",
    "customer_features": [
        "客戶A,交易量大，交易量約1,200萬，喜歡追求風險，大多投資在大中華地區的股票。",
        "客戶B,交易量很小，過去一年僅交易一筆美債，不喜歡風險。",
    ]
}

# 格式化輸出函數，將每個客戶的結果格式化為易於閱讀的文本
def format_output_for_customer(customer_name, features, summary, advice):
    return f"### {customer_name}\n\n" \
           f"**客戶特徵摘要:**\n{features}\n\n" \
           f"**個性化新聞摘要:**\n{summary}\n\n" \
           f"**未來投資建議:**\n{advice}\n"

# 存儲所有客戶的輸出結果
all_customers_data = []

# 單獨處理每個客戶的信息
for i, customer_feature in enumerate(state["customer_features"]):
    # 重置狀態中的結果部分以避免之前的結果混入
    state["extracted_features"] = ""
    state["news_summary"] = ""
    state["investment_advice_result"] = ""

    # 提取單個客戶特徵
    state["customer_features"] = [customer_feature]

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 儲存每個客戶的結果
    all_customers_data.append({
        "客戶": f"客戶{i + 1}",
        "客戶特徵": result["extracted_features"],
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"]
    })

# 生成表格的部分
import pandas as pd

# 使用pandas創建DataFrame並顯示表格
df = pd.DataFrame(all_customers_data)
print(df)


提取的客戶特徵: 客戶A特徵摘要：

* **交易量大：** 約1,200萬
* **風險偏好：**  追求風險
* **投資偏好：**  大中華地區股票
生成的新聞摘要: 客戶B特徵摘要：

* **交易量小：** 約100萬
* **風險偏好：**  保守
* **投資偏好：**  債券

## 新聞摘要:

**針對客戶A：** 近期日圓走強，日經指數創半年新低，但亞洲其他股市表現相對穩定。台股方面，台積電*(2330)等電子股近期出現回調，但長期基本面仍然看好。

建議您：

* **關注日圓走勢:** 日圓升值可能影響日本出口企業獲利，進而影響相關台股供應鏈。
* **逢低佈局科技股:**  科技股短期面臨獲利調節賣壓，但長期成長趨勢不變，可逢低佈局。

**針對客戶B：** 全球經濟衰退風險上升，避險情緒升溫，推升美元及日圓走強。聯準會9月降息預期升溫，利多美債等高評級債券。

建議您：

* **增持美元及日圓資產:** 可考慮配置美元計價債券或日圓ETF，以規避風險。
* **加碼防禦性資產:**  醫療、公用事業等防禦類股受經濟波動影響較小，可考慮加碼配置。
生成的投資建議: **免責聲明：** 以上僅為個人建議，不構成任何投資邀約或承諾。投資前請謹慎評估自身風險承受能力，並諮詢專業理財顧問。
提取的客戶特徵: 客戶B特徵：
* **交易量小**: 過去一年僅交易一筆。
* **風險偏好低**: 不喜歡風險。
* **投資產品**: 美債。
生成的新聞摘要: 新聞摘要：

近期日圓走強，主要受日本央行意外升息及美國經濟衰退擔憂影響。儘管日本央行朝貨幣政策正常化邁進，但全球經濟放緩憂慮加劇避險情緒，推升日圓。由於美國經濟數據疲軟，市場預期聯準會將降息，利好美債等高評級債券。建議客戶B可考慮增持美債，並持有部分現金以降低投資組合波動。
生成的投資建議: ## 給客戶B的投資建議：

根據您的低風險偏好和近期市場動態，建議您：

1. **繼續持有美債**: 美國經濟放緩預期利好美債，符合您的低風險偏好。
2. **考慮增持部分美債**:  可將部分現金用於增持美債，以獲得更穩健的收益。
3. **保持部分現金**:  保留部分現金可應對市場波動，並在出現更佳投資機會時靈活操作。
    客戶  \
0  客戶1   
1  客戶2   

      

In [14]:
# 生成表格的部分
import pandas as pd

# 使用pandas創建DataFrame並顯示表格
df = pd.DataFrame(all_customers_data)
df

,客戶,客戶特徵,新聞摘要,投資建議
0,客戶1,"客戶A特徵摘要：\n\n* **交易量大：** 約1,200萬\n* **風險偏好：** 追求風險\n* **投資偏好：** 大中華地區股票",客戶B特徵摘要：\n\n* **交易量小：** 約100萬\n* **風險偏好：** 保守\n* **投資偏好：** 債券\n\n## 新聞摘要:\n\n**針對客戶A：** 近期日圓走強，日經指數創半年新低，但亞洲其他股市表現相對穩定。台股方面，台積電*(2330)等電子股近期出現回調，但長期基本面仍然看好。\n\n建議您：\n\n* **關注日圓走勢:** 日圓升值可能影響日本出口企業獲利，進而影響相關台股供應鏈。\n* **逢低佈局科技股:** 科技股短期面臨獲利調節賣壓，但長期成長趨勢不變，可逢低佈局。\n\n**針對客戶B：** 全球經濟衰退風險上升，避險情緒升溫，推升美元及日圓走強。聯準會9月降息預期升溫，利多美債等高評級債券。\n\n建議您：\n\n* **增持美元及日圓資產:** 可考慮配置美元計價債券或日圓ETF，以規避風險。\n* **加碼防禦性資產:** 醫療、公用事業等防禦類股受經濟波動影響較小，可考慮加碼配置。,**免責聲明：** 以上僅為個人建議，不構成任何投資邀約或承諾。投資前請謹慎評估自身風險承受能力，並諮詢專業理財顧問。
1,客戶2,客戶B特徵：\n* **交易量小**: 過去一年僅交易一筆。\n* **風險偏好低**: 不喜歡風險。\n* **投資產品**: 美債。,新聞摘要：\n\n近期日圓走強，主要受日本央行意外升息及美國經濟衰退擔憂影響。儘管日本央行朝貨幣政策正常化邁進，但全球經濟放緩憂慮加劇避險情緒，推升日圓。由於美國經濟數據疲軟，市場預期聯準會將降息，利好美債等高評級債券。建議客戶B可考慮增持美債，並持有部分現金以降低投資組合波動。,## 給客戶B的投資建議：\n\n根據您的低風險偏好和近期市場動態，建議您：\n\n1. **繼續持有美債**: 美國經濟放緩預期利好美債，符合您的低風險偏好。\n2. **考慮增持部分美債**: 可將部分現金用於增持美債，以獲得更穩健的收益。\n3. **保持部分現金**: 保留部分現金可應對市場波動，並在出現更佳投資機會時靈活操作。


In [16]:
import pandas as pd

# 假設 all_customers_data 是從之前的結果中獲得的
all_customers_data = [
    {
        "客戶": "客戶1",
        "客戶特徵": "客戶A特徵摘要：\n**交易量大：** 約1,200萬\n**風險偏好：** 喜歡追求風險\n**投資偏好：** 大中華地區股票",
        "新聞摘要": "新聞摘要：\n近期日圓走強，日經指數創半年新低，台股大幅下挫。建議關注美債與高評級債券。\n",
        "投資建議": "投資建議：\n**短期建議：** 減持部分科技股，增持避險資產如美債和黃金。\n**長期建議：** 分散投資，增加多元資產配置。"
    },
    {
        "客戶": "客戶2",
        "客戶特徵": "客戶B特徵摘要：\n**交易量小：** 過去一年僅交易一筆美債。\n**風險偏好低：** 不喜歡風險。",
        "新聞摘要": "新聞摘要：\n全球經濟衰退風險上升，美國經濟數據表現不佳，建議持續關注防禦性資產。",
        "投資建議": "投資建議：\n**低風險建議：** 保持現有美債配置，觀察市場變化，適時調整資產組合。"
    }
]

# 創建 DataFrame
df = pd.DataFrame(all_customers_data)

# 設置顯示選項
pd.set_option('display.max_columns', None)  # 顯示所有列
pd.set_option('display.max_rows', None)  # 顯示所有行
pd.set_option('display.max_colwidth', None)  # 不截斷列的內容

# 輸出結果到Excel文件
df.to_excel("customer_analysis_output.xlsx", index=False)

# 顯示DataFrame
print(df)

    客戶                                                              客戶特徵  \
0  客戶1  客戶A特徵摘要：\n**交易量大：** 約1,200萬\n**風險偏好：** 喜歡追求風險\n**投資偏好：** 大中華地區股票   
1  客戶2               客戶B特徵摘要：\n**交易量小：** 過去一年僅交易一筆美債。\n**風險偏好低：** 不喜歡風險。   

                                             新聞摘要  \
0  新聞摘要：\n近期日圓走強，日經指數創半年新低，台股大幅下挫。建議關注美債與高評級債券。\n   
1       新聞摘要：\n全球經濟衰退風險上升，美國經濟數據表現不佳，建議持續關注防禦性資產。   

                                                               投資建議  
0  投資建議：\n**短期建議：** 減持部分科技股，增持避險資產如美債和黃金。\n**長期建議：** 分散投資，增加多元資產配置。  
1                       投資建議：\n**低風險建議：** 保持現有美債配置，觀察市場變化，適時調整資產組合。  
